# Stacking of Models

In [4]:
import numpy as np
import os
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GroupKFold
import lightgbm as lgb

# 1JHC

In [9]:
oof = pd.read_parquet('../type_results/1JHC/M037_0706_2029_1JHC_oof_0.5959MAE_-0.5176LMAE_catboost_2folds_500000iter_0.1lr.parquet')
oof = oof.drop('oof_preds', axis=1)
for o in [f for f in os.listdir('../type_results/1JHC') if 'oof' in f]:
    df = pd.read_parquet(f'../type_results/1JHC/{o}')
    model_num = o[:4]
    oof[model_num] = df['oof_preds']
    
sub = pd.read_parquet('../type_results/1JHC/M037_0706_2029_1JHC_oof_0.5959MAE_-0.5176LMAE_catboost_2folds_500000iter_0.1lr.parquet')
sub = oof.drop('scalar_coupling_constant', axis=1)
for o in [f for f in os.listdir('../type_results/1JHC') if 'oof' in f]:
    df = pd.read_parquet(f'../type_results/1JHC/{o}')
    model_num = o[:4]
    oof[model_num] = df['oof_preds']

In [11]:
train = pd.read_csv('../input/train.csv')

In [12]:
oof = pd.merge(train[['id','molecule_name']],
          oof,
         on=['id']
        )

In [13]:
X = oof[[m for m in oof.columns if 'M' in m]]
y = oof['scalar_coupling_constant']
G = oof['molecule_name']

In [24]:
y_valid.head()

0     84.8076
1     84.8074
2     84.8093
3     84.8095
21    88.2837
Name: scalar_coupling_constant, dtype: float64

In [21]:
folds = GroupKFold(n_splits=3)
for fold_n, (train_idx, valid_idx) in enumerate(folds.split(X, groups=G)):
    X_train = X.iloc[train_idx]
    X_valid = X.iloc[valid_idx]
    y_train = y.iloc[train_idx]
    y_valid = y.iloc[valid_idx]
    reg = lgb.LGBMRegressor(n_estimators=1000, n_jobs=20)
    reg.fit(X_train, y_train,
            eval_set=(X_valid, y_valid),
            verbose=100)
    oof['stack_oof'] = reg.predict(X)

#reg = lgb.LGBMRegressor(n_estimators=5000, n_jobs=20)
# reg.fit(X, y)
oof['stack_oof'] = reg.predict(X)

[100]	valid_0's l2: 1.7989
[200]	valid_0's l2: 1.79541
[300]	valid_0's l2: 1.79553
[400]	valid_0's l2: 1.79493
[500]	valid_0's l2: 1.79583
[600]	valid_0's l2: 1.79561
[700]	valid_0's l2: 1.79548
[800]	valid_0's l2: 1.79598
[900]	valid_0's l2: 1.79637
[1000]	valid_0's l2: 1.79666


KeyboardInterrupt: 

In [47]:
mean_absolute_error(oof['scalar_coupling_constant'], oof['stack_oof'])

0.47947805603785626

In [49]:
mean_absolute_error(oof['scalar_coupling_constant'], oof['stack_oof'])

0.4344068243707282

In [ ]:
reg.predict('')